Project notebook for applied_statistics

This notebook is designed to analyse the PlantGrowth dataset and perform statistical analysis on it using t-test and ANOVA methods.
The dataset was downloaded from https://github.com/vincentarelbundock/Rdatasets/blob/master/csv/datasets/PlantGrowth.csv and contains results of an experiment to compare the weights of plants under three different conditions.


In [35]:
# Import any required modules

import pandas as pd

In [36]:
# Import the data into a dataframe 

df = pd.read_csv('PlantGrowth.csv')

df.head()

,rownames,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl
2,3,5.18,ctrl
3,4,6.11,ctrl
4,5,4.50,ctrl


Describe the data

In [3]:
df.describe()

,rownames,weight
count,30.000000,30.000000
mean,15.500000,5.073000
std,8.803408,0.701192
min,1.000000,3.590000
25%,8.250000,4.550000
50%,15.500000,5.155000
75%,22.750000,5.530000
max,30.000000,6.310000


In [5]:
df.shape

(30, 3)

In [6]:
df.info

<bound method DataFrame.info of     rownames  weight group
0          1    4.17  ctrl
1          2    5.58  ctrl
2          3    5.18  ctrl
3          4    6.11  ctrl
4          5    4.50  ctrl
5          6    4.61  ctrl
6          7    5.17  ctrl
7          8    4.53  ctrl
8          9    5.33  ctrl
9         10    5.14  ctrl
10        11    4.81  trt1
11        12    4.17  trt1
12        13    4.41  trt1
13        14    3.59  trt1
14        15    5.87  trt1
15        16    3.83  trt1
16        17    6.03  trt1
17        18    4.89  trt1
18        19    4.32  trt1
19        20    4.69  trt1
20        21    6.31  trt2
21        22    5.12  trt2
22        23    5.54  trt2
23        24    5.50  trt2
24        25    5.37  trt2
25        26    5.29  trt2
26        27    4.92  trt2
27        28    6.15  trt2
28        29    5.80  trt2
29        30    5.26  trt2>

In [13]:
unique_class = df.group.unique()
unique_class

array(['ctrl', 'trt1', 'trt2'], dtype=object)

In [46]:
# Cycle through the three groups and print summary stats for each one

grp=0
while grp < 3:
     data = df.loc[df["group"]==unique_class[grp]]
     data = data.describe().weight[['min', 'max', 'mean', 'std']]
     print(f"Group: \t{unique_class[grp]}, \n{data}\n")
     grp=grp+1

Group: 	ctrl, 
min     4.170000
max     6.110000
mean    5.032000
std     0.583091
Name: weight, dtype: float64

Group: 	trt1, 
min     3.590000
max     6.030000
mean    4.661000
std     0.793676
Name: weight, dtype: float64

Group: 	trt2, 
min     4.920000
max     6.310000
mean    5.526000
std     0.442573
Name: weight, dtype: float64



t-test - what it is, how it works and assumptions

Perform t-test

ANOVA test - description

Perform ANOVA test

Explain why ANOVA is more appropriate when analysinng more than two groups